In [3]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from skimage.filters import unsharp_mask
from skimage.metrics import structural_similarity as ssim


# Load the saved autoencoder model
autoencoder= load_model(r'C:\Users\USER\Desktop\Deblurring\Model_Training\model(3).h5')

def is_blurred(image, threshold=100):
    # You need to define a suitable threshold based on your model and dataset
    # The threshold will depend on how confident your model needs to be to classify an image as blurred
    # It should be a value between 0 and 1
   # image = image / 255.0  # Normalize pixel values to [0, 1]
    image = cv2.resize(image, (128, 128))  # Assuming your model expects input size (128, 128, 3)
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    # Use the autoencoder model to predict
    prediction = autoencoder.predict(image)

    # You can use the mean squared error or other metrics to decide if the image is blurred
    mse = np.mean(np.square(image - prediction))
    
    
    
    return mse > threshold  # Adjust threshold as needed

def deblur_image(image_path):
    image=tf.keras.preprocessing.image.load_img(image_path, target_size=(128,128))
    image=tf.keras.preprocessing.image.img_to_array(image).astype('float32') / 255
    predict=image.reshape(1,128,128,3)
    deblurred_image=autoencoder.predict(predict)
    deblurred_image=deblurred_image.reshape(128,128,3)

    

    return deblurred_image

# Example usage:
image_path = r"C:\Users\USER\Desktop\Deblurring\data\defocused_blurred\16_IPHONE-7_F.jpeg"

input_image = cv2.imread(image_path)


if is_blurred(input_image):
    # If the image is blurred, deblur it
    deblurred_image = deblur_image(image_path)
    deblurred_image = (deblurred_image * 255).astype(np.uint8)

    
    grayA = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    deblurred_image_resized = cv2.resize(deblurred_image, (grayA.shape[1], grayA.shape[0]))
    grayB= cv2.cvtColor(deblurred_image_resized, cv2.COLOR_BGR2GRAY)   
    (score, diff) = ssim(grayA, grayB, full=True)
    diff = (diff * 255).astype("uint8")
    print("SSIM: {}".format(score))          

    # Display the original, blurred, and deblurred images for comparison
    cv2.imshow("Original", input_image)
    cv2.imwrite('Orginal_image.jpeg',input_image)
    cv2.imshow("Blurred", input_image)
    cv2.imwrite('Blurred_image.jpeg',input_image)  # Change to your blurred image if you have one
    cv2.imshow("Deblurred", deblurred_image)
    cv2.imwrite('Deblurred_image.jpeg',deblurred_image)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    # If the image is not blurred, do nothing or display the original image
    cv2.imshow("Original", input_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 143ms/step
SSIM: 0.9150685386304823
